In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
import plotly
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import plotly.express as px
from pathlib import Path
import warnings
import config
import preprocess
from custom_metrics import LoglossObjective_loop, LoglossObjective_np
from collections import defaultdict
from catboost import CatBoostClassifier, Pool

sys.path.append('./cython_loss')
from cython1 import LoglossObjective_cython

os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
# import featuretools as ft

warnings.filterwarnings("ignore", category=DeprecationWarning)

pd.set_option('display.max_rows', 120)
plt.style.use("dark_background")
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams["hist.bins"] = 50
print('cur_dir', Path.cwd())

cur_dir /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Otus/ML_advanced2020/final_project


In [2]:
data = pd.read_csv("/home/sergey/mnt/4.5Tb/Downloads/otus_final_input/data_encoded.csv",
                   low_memory=False,
                   ).astype(config.dtypes)
data = preprocess.preprocess(data)
print("", data.shape)
data.head()

/home/sergey/anaconda3/envs/final_project/lib/python3.8/site-packages/featuretools_update_checker/functions.py:34: UserWarning: Featuretools is out-of-date: installed == 0.23.1, latest == 0.23.2
  warnings.warn(msg)


 (274702, 13)


,DealDurationDays,DealDurationMonths,Client_Id,DealDate,ValueDate,MaturityDate,Deal_characteristics_1,Deal_characteristics_2,Deal_characteristics_3,Deal_characteristics_4,Client_characteristics_1,Client_characteristics_2,target
0,8.0,0,16140,2015-01-12,2015-01-12,2015-01-20,7.018746,0.0000,0.140368,0,15,1759,1
1,53.0,1,21990,2015-01-12,2015-01-12,2015-03-06,0.306237,0.1146,0.087720,2,15,1759,1
2,92.0,3,18607,2015-01-12,2015-01-12,2015-04-14,0.306237,0.1201,0.054139,2,15,1759,1
3,1.0,0,20500,2015-01-12,2015-01-12,2015-01-13,2.985808,0.0896,0.100056,2,15,1759,1
4,1.0,0,4041,2015-01-12,2015-01-12,2015-01-13,0.418013,0.0805,0.168161,2,15,1759,1


In [3]:
# print(data.columns)

In [4]:
X_val = data.groupby(data.Client_Id).tail(1)
y_val = X_val.pop('target')
X_train = data.drop(labels=X_val.index)
y_train = X_train.pop('target')

In [5]:
client_id_activity = X_train.groupby(data.Client_Id).agg(['count', 'mean', 'median'])
print('активность клиентов, простые статистики')
client_id_activity.sort_values(by=('DealDurationDays', 'count'), ascending=False)

активность клиентов, простые статистики


DealDurationDays                   DealDurationMonths            \
                     count       mean median              count      mean   
Client_Id                                                                   
0                    16510  13.290612    7.0              16510  0.245548   
7596                  3353   1.526991    1.0               3353  0.000000   
17708                 2158   1.994903    1.0               2158  0.002780   
8269                  1797   5.678353    6.0               1797  0.000000   
11790                 1439  29.776928   26.0               1439  0.503127   
...                    ...        ...    ...                ...       ...   
1476                     1  28.000000   28.0                  1  0.000000   
10884                    1   7.000000    7.0                  1  0.000000   
1462                     1   3.000000    3.0                  1  0.000000   
5381                     1  13.000000   13.0                  1  0.000000   
6840                     1  33.000000   33.0                  1  1.000000   

                 Client_Id               Deal_characteristics_1  ...  \
          median     count   mean median                  count  ...   
Client_Id                                                        ...   
0            0.0     16510      0      0                  16510  ...   
7596         0.0      3353   7596   7596                   3353  ...   
17708        0.0      2158  17708  17708                   2158  ...   
8269         0.0      1797   8269   8269                   1797  ...   
11790        0.0      1439  11790  11790                   1439  ...   
...          ...       ...    ...    ...                    ...  ...   
1476         0.0         1   1476   1476                      1  ...   
10884        0.0         1  10884  10884                      1  ...   
1462         0.0         1   1462   1462                      1  ...   
5381         0.0         1   5381   5381                      1  ...   
6840         1.0         1   6840   6840                      1  ...   

          Deal_characteristics_3 Deal_characteristics_4                   \
                          median                  count      mean median   
Client_Id                                                                  
0                       0.074074                  16510  1.677832      2   
7596                    0.067925                   3353  2.000000      2   
17708                   0.076577                   2158  1.000000      1   
8269                    0.080000                   1797  0.000000      0   
11790                   0.079261                   1439  0.000000      0   
...                          ...                    ...       ...    ...   
1476                    0.127649                      1  0.000000      0   
10884                   0.109683                      1  2.000000      2   
1462                    0.120395                      1  2.000000      2   
5381                    0.078762                      1  2.000000      2   
6840                    0.086843                      1  2.000000      2   

          Client_characteristics_1                   Client_characteristics_2  \
                             count       mean median                    count   
Client_Id                                                                       
0                            16510  10.497638   13.0                    16510   
7596                          3353   7.232627    7.0                     3353   
17708                         2158  15.000000   15.0                     2158   
8269                          1797  10.648303   15.0                     1797   
11790                         1439  10.970813   15.0                     1439   
...                            ...        ...    ...                      ...   
1476                             1  15.000000   15.0                        1   
10884                            1  15.000000  

In [6]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)

    X_train = X_train.merge(client_id_activity, how='left', left_on='Client_Id', right_index=True).dropna()
    X_val = X_val.merge(client_id_activity, how='left', left_on='Client_Id', right_index=True)  # .dropna()
# y_val = y_val[X_val.index]

In [7]:
print('data.shape', data.shape)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('data.shape[0] - X_val.shape[0] = ', data.shape[0] - X_val.shape[0])
print('data.shape[0] -  X_train.shape[0] - X_val.shape[0] = ', data.shape[0] - X_train.shape[0] - X_val.shape[0])
print(X_train.Client_Id.nunique(), X_val.Client_Id.nunique(), X_train.Client_Id.nunique() - X_val.Client_Id.nunique())

data.shape (274702, 13)
X_train.shape (257180, 39)
X_val.shape (17522, 39)
data.shape[0] - X_val.shape[0] =  257180
data.shape[0] -  X_train.shape[0] - X_val.shape[0] =  0
13050 17522 -4472


In [8]:
train_pool = Pool(data=X_train, label=y_train,
                  cat_features=config.categorical_dtypes_ft.keys())
val_pool = Pool(data=X_val, label=y_val,
                cat_features=config.categorical_dtypes_ft.keys())

In [9]:
%%time
params = config.params
params.update({
    'iterations': 20,
    'verbose': 1,
})
clf_logloss_ref = CatBoostClassifier(**params)
clf_logloss_ref.fit(X=train_pool,
        eval_set=val_pool,
        use_best_model=True)


0:	learn: 0.7196265	test: 0.6101332	best: 0.6101332 (0)	total: 291ms	remaining: 5.53s
1:	learn: 0.7196265	test: 0.6101332	best: 0.6101332 (0)	total: 562ms	remaining: 5.05s
2:	learn: 0.7196265	test: 0.6101332	best: 0.6101332 (0)	total: 784ms	remaining: 4.44s
3:	learn: 0.7246614	test: 0.6145145	best: 0.6145145 (3)	total: 957ms	remaining: 3.83s
4:	learn: 0.7246614	test: 0.6145145	best: 0.6145145 (3)	total: 1.16s	remaining: 3.49s
5:	learn: 0.7246614	test: 0.6145145	best: 0.6145145 (3)	total: 1.35s	remaining: 3.16s
6:	learn: 0.7246614	test: 0.6145145	best: 0.6145145 (3)	total: 1.61s	remaining: 2.99s
7:	learn: 0.7246614	test: 0.6145145	best: 0.6145145 (3)	total: 1.83s	remaining: 2.75s
8:	learn: 0.7252500	test: 0.6145384	best: 0.6145384 (8)	total: 2.1s	remaining: 2.56s
9:	learn: 0.7252500	test: 0.6145384	best: 0.6145384 (8)	total: 2.35s	remaining: 2.35s
10:	learn: 0.7251849	test: 0.6145384	best: 0.6145384 (8)	total: 2.54s	remaining: 2.08s
11:	learn: 0.7251849	test: 0.6145384	best: 0.6145384 (

In [12]:
# %%time
# params = config.params
# params.update({
#     'loss_function': LoglossObjective_np()
# })
# clf_logloss_custom_np = CatBoostClassifier(**params)
# clf_logloss_custom_np.fit(X=train_pool,
#         eval_set=val_pool,
#         use_best_model=True)
# print(f'Функция потерь logloss реализована через numpy, \n'
#       f'примерно в 4 раза медленнее, чем родная от catboost \n'
#       f'но примерно в 8 раз быстрее, чем если делать с циклом внутри \n'
#       f'также сходится медленнее, чем родная. Сходимость одинаковая с реализацией через цикл \n'
#       f'о сходимости есть топик на stackoverflow здесь \n'
#       f'https://stackoverflow.com/questions/63104119/catboost-custom-loss-function \n'
#       f'')

In [13]:
# %%time
# params = config.params
# params.update({
#     'loss_function': LoglossObjective_loop()
# })
# clf_logloss_custom_loop = CatBoostClassifier(**params)
# clf_logloss_custom_loop.fit(X=train_pool,
#         eval_set=val_pool,
#         use_best_model=True)
# print(f'Здесь функция потерь logloss реализована через цикл, очень медленно выполняется, \n'
#       f'примерно в 8 раз медленнее, чем если реализовать через numpy \n'
#       f'и в 30 раз медленнее, чем референсная, родная catboost \n'
#       f'также сходится медленнее, чем родная. Сходимость одинаковая с реализацией через numpy ')

In [14]:
import sys, os
sys.path.append('./cython_loss')
from cython1 import LoglossObjective_cython

In [15]:
%%time
params = config.params
params.update({
    'loss_function': LoglossObjective_cython()
})
clf_logloss_custom_loop = CatBoostClassifier(**params)
clf_logloss_custom_loop.fit(X=train_pool,
        eval_set=val_pool,
        use_best_model=True)
print(f'Здесь функция потерь logloss реализована через цикл, очень медленно выполняется, \n'
      f'примерно в 8 раз медленнее, чем если реализовать через numpy \n'
      f'и в 30 раз медленнее, чем референсная, родная catboost \n'
      f'также сходится медленнее, чем родная. Сходимость одинаковая с реализацией через numpy ')


CatBoostError: catboost/python-package/catboost/helpers.cpp:42: Traceback (most recent call last):
  File "_catboost.pyx", line 1343, in _catboost._ObjectiveCalcDersRange
TypeError: Argument 'approxes' has incorrect type (expected numpy.ndarray, got _catboost._DoubleArrayWrapper)


In [16]:
print('clf_logloss_ref.get_evals_result()')
ref_res = clf_logloss_ref.get_evals_result()
ref_res

clf_logloss_ref.get_evals_result()


{'learn': {'F1': [0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422,
   0.8191941399415422],
  'Logloss': [0.6924837042047295,
   0.6918232026721597,
   0.6911656612837891,
   0.6905378603853575,
   0.6898860108653373,
   0.6892637369665031,
   0.6886175260068954,
   0.6880007249463429,
   0.687361014207238,
   0.68672323289159,
   0.6861145302441155,
   0.6854822533375775,
   0.6848888296184447,
   0.684261923605289,
   0.6836386836043069,
   0.6830523178062149,
   0.682433499522553,
   0.6818528763181778,
   0.6812393091013396,
   0.6806289146465265],
  'AUC': [0.7196265386422204,
   0.7196265386422204,
   0

In [17]:
print('clf_logloss_custom_np.get_evals_result()')
np_res = clf_logloss_custom_np.get_evals_result()
np_res

clf_logloss_custom_np.get_evals_result()


NameError: name 'clf_logloss_custom_np' is not defined

In [ ]:
print('clf_logloss_custom_loop.get_evals_result()')
loop_res = clf_logloss_custom_loop.get_evals_result()
loop_res

In [ ]:
print('FINISH')